# Run MPNN multistate design on the paired states

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/projects/crispy_shifty/projects/crispy_shifties
running on node: dig90


### Set working directory to the root of the crispy_shifty repo

In [2]:
os.chdir("/projects/crispy_shifty")

### Run MPNN on the paired states

In [3]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "06_mpnn_paired_states"
design_list_file = os.path.join(
    os.getcwd(),
    "projects/crispy_shifties/05_design_paired_states/designed_paired_states.list",
)
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

options = " ".join(
    [
        "out:level 200",
    ]
)

extra_kwargs = {
    "num_sequences": "100",
    "mpnn_temperature": "0.2",
    "mpnn_design_area": "scan",
}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.mpnn.mpnn_paired_state",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="short",
    memory="5G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


Run the following command with your desired environment active:
sbatch -a 1-45268 /mnt/projects/crispy_shifty/projects/crispy_shifties/06_mpnn_paired_states/run.sh


### Collect scorefiles of the MPNN designed paired states and concatenate

In [4]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "06_mpnn_paired_states"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile

In [5]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import parse_scorefile_linear

output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))

### Dump scores_df as a CSV and then reload, for performance reasons

In [6]:
if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df.to_csv(os.path.join(output_path, "scores.csv"))

scores_df = pd.read_csv(os.path.join(output_path, "scores.csv"), index_col="Unnamed: 0")

### Data exploration

In [7]:
print(len(scores_df))
print(list(scores_df.columns))

135777
['bb_clash', 'bb_clash_delta', 'bb_clash_delta_x', 'best_average_DAN_plddts', 'best_average_plddts', 'best_model', 'best_ptm', 'best_rmsd_to_input', 'buns_parent', 'cms_AcB', 'cms_AnAc', 'cms_AnAcB', 'cms_AnB', 'designed_by', 'docked_helix', 'dslf_fa13', 'dssp', 'elapsed_time', 'exposed_hydrophobics_parent', 'fa_atr', 'fa_dun', 'fa_dun_dev', 'fa_dun_rot', 'fa_dun_semi', 'fa_elec', 'fa_intra_atr_xover4', 'fa_intra_elec', 'fa_intra_rep', 'fa_intra_rep_xover4', 'fa_intra_sol_xover4', 'fa_rep', 'fa_sol', 'geometry_parent', 'hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb', 'holes_all_parent', 'holes_core_parent', 'hxl_tors', 'interface', 'lk_ball', 'lk_ball_bridge', 'lk_ball_bridge_uncpl', 'lk_ball_iso', 'lk_ball_wtd', 'loop_dist', 'loop_sc', 'looped_length', 'mean_pae', 'mean_pae_interaction', 'mean_pae_interaction_AB', 'mean_pae_interaction_BA', 'mean_pae_intra_chain', 'mean_pae_intra_chain_A', 'mean_pae_intra_chain_B', 'mean_plddt', 'mismatch_probability_parent', 'model', '

In [8]:
set(scores_df.mpnn_msd_design_area.values)

{'full', 'interface', 'neighborhood'}

### Rebalance the df

In [9]:
rebalanced = []
goal_representation = 32

for state in tqdm(set(scores_df.state.values)):
    subset_df = scores_df.query(f"state == @state")
    # if len is less than or equal to goal_representation take all,
    if len(subset_df) <= goal_representation:
        rebalanced.extend(list(subset_df.index))
    else:
        # get n randomly selected from this state
        sample = subset_df.sample(goal_representation, random_state=0)
        rebalanced.extend(list(sample.index))
rebalanced_df = scores_df.loc[rebalanced]
len(rebalanced_df)

  0%|          | 0/777 [00:00<?, ?it/s]

22130

### Save individual fastas

In [10]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import df_to_fastas

output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

rebalanced_df = df_to_fastas(rebalanced_df, prefix="mpnn_seq")

  0%|          | 0/22130 [00:00<?, ?it/s]

### Save a list of outputs
Sort by length

In [11]:
simulation_name = "06_mpnn_paired_states"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

scores_df = rebalanced_df.sort_values("looped_length")

with open(os.path.join(output_path, "mpnn_paired_states.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

  0%|          | 0/22130 [00:00<?, ?it/s]

### Concat the pdb.bz2 and fasta paths into a single list, for Superfold reasons

In [12]:
simulation_name = "06_mpnn_paired_states"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

with open(os.path.join(output_path, "mpnn_paired_states.pair"), "w") as f:
    for path in tqdm(scores_df.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

  0%|          | 0/22130 [00:00<?, ?it/s]

### Run MPNN on the best paired states

In [13]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "06_mpnn_paired_states_best"
design_list_file = os.path.join(
    os.getcwd(),
    "projects/crispy_shifties/05_design_paired_states_best/best_designed_paired_states.list",
)
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

options = " ".join(
    [
        "out:level 200",
    ]
)

extra_kwargs = {
    "num_sequences": "100",
    "mpnn_temperature": "0.2",
    "mpnn_design_area": "scan",
}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.mpnn.mpnn_paired_state",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="short",
    memory="5G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

Run the following command with your desired environment active:
sbatch -a 1-3940 /mnt/projects/crispy_shifty/projects/crispy_shifties/06_mpnn_paired_states_best/run.sh


### Collect scorefiles of the MPNN designed paired states and concatenate

In [3]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "06_mpnn_paired_states_best"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


### Load resulting concatenated scorefile

In [4]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import parse_scorefile_linear

output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))

  0%|          | 0/11820 [00:00<?, ?it/s]

### Dump scores_df as a CSV and then reload, for performance reasons

In [5]:
if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df.to_csv(os.path.join(output_path, "scores.csv"))

scores_df = pd.read_csv(os.path.join(output_path, "scores.csv"), index_col="Unnamed: 0")

### Rebalance the df

In [8]:
rebalanced = []
goal_representation = 50

for state in tqdm(set(scores_df.state.values)):
    subset_df = scores_df.query(f"state == @state")
    # if len is less than or equal to goal_representation take all,
    if len(subset_df) <= goal_representation:
        rebalanced.extend(list(subset_df.index))
    else:
        # get n randomly selected from this state
        sample = subset_df.sample(goal_representation, random_state=0)
        rebalanced.extend(list(sample.index))
rebalanced_df = scores_df.loc[rebalanced]
len(rebalanced_df)

  0%|          | 0/129 [00:00<?, ?it/s]

6352

### Save individual fastas

In [9]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import df_to_fastas

output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

rebalanced_df = df_to_fastas(rebalanced_df, prefix="mpnn_seq")

  0%|          | 0/6352 [00:00<?, ?it/s]

### Save a list of outputs
Sort by length

In [10]:
simulation_name = "06_mpnn_paired_states_best"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

scores_df = rebalanced_df.sort_values("looped_length")

with open(os.path.join(output_path, "best_mpnn_paired_states.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

  0%|          | 0/6352 [00:00<?, ?it/s]

### Concat the pdb.bz2 and fasta paths into a single list, for Superfold reasons

In [11]:
simulation_name = "06_mpnn_paired_states_best"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

with open(os.path.join(output_path, "best_mpnn_paired_states.pair"), "w") as f:
    for path in tqdm(scores_df.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

  0%|          | 0/6352 [00:00<?, ?it/s]

### Prototyping blocks

test `mpnn_paired_state`

In [ ]:
%%time 
import pyrosetta

pyrosetta.init()


sys.path.insert(0, "/projects/crispy_shifty/")
from crispy_shifty.protocols.mpnn import mpnn_paired_state

t = mpnn_paired_state(
        None,
        **{
            'pdb_path': '/mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/05_design_paired_states/decoys/0000/05_design_paired_states_c6be6ebc8a3146e2960cb45360a8a202.pdb.bz2',
            'num_sequences': 100,
        }
)
for i, tppose in enumerate(t):
    tppose.pose.dump_pdb(f"{i}.pdb")

In [ ]:
d = dict(tppose.pose.scores)
#
d